In [ ]:
import os
import json
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from sklearn.metrics import mean_squared_error
import logging
import re
from tqdm import tqdm

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 경로 및 하이퍼파라미터 설정
MODEL_PATH = "olmo7B-v12-80000"
OUTPUT_DIR = "olmo7B-klue-sts"
DATA_CACHE_DIR = "./klue_sts_cache"
JSON_DATASET_PATH = "./klue_sts.json"
MAX_LENGTH = 512
MAX_EVAL_SAMPLES = 200

# 데이터셋 준비 함수 - JSON 파일 생성
def prepare_dataset_json():
    """KLUE STS 데이터셋을 불러와서 JSON 파일로 변환합니다."""
    if os.path.exists(JSON_DATASET_PATH):
        logger.info(f"Dataset already exists: {JSON_DATASET_PATH}")
        return
    
    logger.info("KLUE STS dataset loading...")
    klue_sts = load_dataset("klue", "sts", cache_dir=DATA_CACHE_DIR)
    
    # 학습 및 검증 데이터를 위한 리스트
    train_samples = []
    val_samples = []
    
    # 함수: 프롬프트와 완성 텍스트 생성
    def create_prompt(sentence1, sentence2):
        return f"Analyze the following sentence pairs and provide a similarity score between 0 and 5, where 0 means completely different and 5 means identical in meaning. Sentence 1: {sentence1} Sentence 2: {sentence2}"

    def create_completion(score):
        # 정수로 반올림된 점수 반환
        return f" The similarity score is {score}"
    
    # 학습 데이터 준비
    logger.info("Creating Klue_dataset.json ...")
    for item in tqdm(klue_sts["train"]):
        sentence1 = item["sentence1"]
        sentence2 = item["sentence2"]
        score = item["labels"]["label"]  # 0-5 척도
        
        # 스코어 정규화
        normalized_score = max(0, min(5, score))
        
        sample = {
            "input": create_prompt(sentence1, sentence2),
            "output": create_completion(normalized_score)
        }
        train_samples.append(sample)
    
    # 검증 데이터 준비 (메모리 절약을 위해 일부만 사용)
    logger.info("Translating Valid data...")
    val_subset = klue_sts["validation"]
    for item in tqdm(val_subset):
        sentence1 = item["sentence1"]
        sentence2 = item["sentence2"]
        score = item["labels"]["label"]
        
        normalized_score = max(0, min(5, score))
        
        sample = {
            "input": create_prompt(sentence1, sentence2),
            "output": create_completion(normalized_score)
        }
        val_samples.append(sample)
    
    # JSON 파일로 저장
    dataset = {
        "train": train_samples,
        "validation": val_samples
    }
    
    logger.info(f"JSON dataset saving... (train: {len(train_samples)}, valid: {len(val_samples)})")
    with open(JSON_DATASET_PATH, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)
    
    logger.info(f"Created klue_sts dataset: {JSON_DATASET_PATH}")

# 데이터 전처리 함수
def preprocess_function(examples, tokenizer, max_length=MAX_LENGTH):
    # 프롬프트 형식
    inputs = tokenizer([ex for ex in examples["input"]], 
                      truncation=True, 
                      max_length=max_length,
                      padding="max_length",
                      return_tensors="pt")
    
    # 출력 토큰화
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([ex for ex in examples["output"]], 
                         truncation=True, 
                         max_length=128,  # 출력은 짧기 때문에 더 작은 길이 사용
                         padding="max_length",
                         return_tensors="pt")
    
    # 라벨 처리: -100은 손실 계산에서 무시됨
    for i in range(len(labels["input_ids"])):
        labels["input_ids"][i][labels["input_ids"][i] == tokenizer.pad_token_id] = -100
    
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"]
    }

# 메인 학습 함수
def train_model():
    # 데이터셋 준비
    prepare_dataset_json()
    
    # 데이터셋 로드
    logger.info("JSON loading...")
    with open(JSON_DATASET_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    train_data = data["train"]
    val_data = data["validation"]
    
    logger.info(f"train data: {len(train_data)}, valid data: {len(val_data)}")
    
    # 모델 및 토크나이저 로드
    logger.info(f"Load model: {MODEL_PATH}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    
    # 특수 토큰 확인 및 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # bfloat16 정밀도로 모델 로드 (메모리 효율성 증가)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.bfloat16,
        device_map="auto"  # 자동으로 GPU에 모델 분산
    )
    
    # 학습용 데이터셋 생성
    from torch.utils.data import Dataset
    
    class SimpleDataset(Dataset):
        def __init__(self, data, tokenizer, max_length):
            self.data = data
            self.tokenizer = tokenizer
            self.max_length = max_length
        
        def __len__(self):
            return len(self.data)
        
        def __getitem__(self, idx):
            item = self.data[idx]
            prompt = item["input"]
            completion = item["output"]
            
            # 프롬프트와 완성 결합
            full_text = prompt + completion
            
            # 토큰화
            encoded = self.tokenizer(
                full_text,
                truncation=True,
                max_length=self.max_length,
                padding="max_length",
                return_tensors="pt"
            )
            
            # 프롬프트 부분 토큰화 (라벨 마스킹용)
            prompt_encoded = self.tokenizer(
                prompt,
                truncation=True,
                max_length=self.max_length,
                return_tensors="pt"
            )
            
            # 라벨 생성: 프롬프트 부분은 -100으로 마스킹
            labels = encoded["input_ids"].clone().squeeze(0)
            prompt_length = prompt_encoded["input_ids"].shape[1]
            labels[:prompt_length] = -100
            
            return {
                "input_ids": encoded["input_ids"].squeeze(0),
                "attention_mask": encoded["attention_mask"].squeeze(0),
                "labels": labels
            }
    
    # 데이터셋 생성
    train_dataset = SimpleDataset(train_data, tokenizer, MAX_LENGTH)
    val_dataset = SimpleDataset(val_data, tokenizer, MAX_LENGTH)
    
    # 데이터 콜레이터
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
    
    # 학습 하이퍼파라미터 설정
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy="steps",
        eval_steps=200,
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=3,
        save_strategy="steps",
        save_steps=400,
        logging_dir="./logs",
        logging_steps=100,
        fp16=False,
        bf16=True,  # bfloat16 정밀도 사용
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        report_to="none",
        gradient_checkpointing=True,  # 메모리 절약을 위한 그래디언트 체크포인팅
        optim="adamw_torch",  # PyTorch 구현 사용
    )
    
    # 얼리 스토핑 콜백
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=3
    )
    
    # 트레이너 초기화 및 학습
    logger.info("Reset Trainer...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        callbacks=[early_stopping_callback],
    )
    
    # 학습 실행
    trainer.train()
    
    # 최종 모델 및 토크나이저 저장
    final_model_path = os.path.join(OUTPUT_DIR, "final")
    logger.info(f"Final Model: {final_model_path}")
    model.save_pretrained(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    
    logger.info("Tuned!")
    return model, tokenizer

# 모델 평가 함수
def evaluate_model(model, tokenizer):
    logger.info("Evaluating the model...")
    
    # KLUE STS 데이터셋 로드
    klue_sts = load_dataset("klue", "sts", cache_dir=DATA_CACHE_DIR)
    
    # 평가용 하위 집합
    val_subset = klue_sts["validation"]
    
    model.eval()
    
    true_scores = []
    pred_scores = []
    

    for item in tqdm(val_subset):
        sentence1 = item["sentence1"]
        sentence2 = item["sentence2"]
        true_score = item["labels"]["label"]
        
        # 프롬프트 생성
        prompt = f"Analyze the following sentence pairs and provide a similarity score between 0 and 5, where 0 means completely different and 5 means identical in meaning. Sentence 1: {sentence1} Sentence 2: {sentence2}"
        
        # 토큰화
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        # 추론
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_new_tokens=20,
                temperature=0.1,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id
            )
        
        # 결과 디코딩
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 점수 추출 (실수 처리)
        match = re.search(r'similarity score is (\d+(\.\d+)?)', generated_text)
        if match:
            predicted_score = float(match.group(1))  # 실수로 변환
            if 0 <= predicted_score <= 5:
                true_scores.append(true_score)
                pred_scores.append(predicted_score)
            else:
                logger.warning(f"Out of scope (0-5): {predicted_score}")
        else:
            logger.warning(f"Can't extract the label: {generated_text}")
    
    # 메트릭 계산
    if true_scores and pred_scores:
        pearson_corr = np.corrcoef(true_scores, pred_scores)[0, 1]
        rmse = np.sqrt(mean_squared_error(true_scores, pred_scores))
        
        logger.info(f"Eval result:")
        logger.info(f"Evaluated samples: {len(true_scores)}")
        logger.info(f"Pearson Correction: {pearson_corr:.4f}")
        logger.info(f"RMSE: {rmse:.4f}")
        
        # 평가 결과 저장
        with open(os.path.join(OUTPUT_DIR, "eval_results.json"), "w") as f:
            json.dump({
                "pearson_correlation": float(pearson_corr),
                "rmse": float(rmse),
                "num_samples": len(true_scores)
            }, f, indent=2)
    else:
        logger.warning("No valid prediction.")

# 메인 실행 함수
if __name__ == "__main__":
    # 출력 디렉토리 생성
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(DATA_CACHE_DIR, exist_ok=True)
    
    # 모델 학습
    model, tokenizer = train_model()
    
    # 모델 평가
    evaluate_model(model, tokenizer)

INFO:__main__:Dataset already exists: ./klue_sts.json
INFO:__main__:JSON loading...
INFO:__main__:train data: 11668, valid data: 519
INFO:__main__:Load model: olmo7B-v12-80000
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/jsong132/.local/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
INFO:__main__:Reset Trainer...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


In [1]:
#######################################################
# KLUE STS dataset
#######################################################
from datasets import load_dataset

klue_sts = load_dataset('klue', 'sts')
   
# 데이터셋 확인 (예: train 데이터셋의 50번째 샘플 확인)
print(klue_sts['train'])  # 또는 'validation' 또는 'test'에 접근 가능

# 예시로 'validation' 데이터셋을 확인
print(klue_sts['validation'])

for i in range(10):
    print(f"{i}th data")
    print("sentence1: ", klue_sts['train'][i]["sentence1"])
    print("sentence2: ", klue_sts['train'][i]["sentence2"])
    print("similarity: ", klue_sts['train'][i]["labels"]["label"])

Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 11668
})
Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 519
})
0th data
sentence1:  숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
sentence2:  숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.
similarity:  3.7
1th data
sentence1:  위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.
sentence2:  시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.
similarity:  0.0
2th data
sentence1:  회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.
sentence2:  사람들이 주로 네이버 메일을 쓰는 이유를 알려줘
similarity:  0.3
3th data
sentence1:  긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업성공패키지, 청년구직활동지원금, 긴급복지지원제도 지원금과는 중복 수급이 불가능하다.
sentence2:  고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.
similarity:  0.6
4th data
sentence1:  호스트의 답장이 늦으나, 개선될 것으로 보입니다.
sentence2:  호스트 응답이 늦었지만 개선될 것으로 보입니다.
similarity:  4.7
5th data
sentence1:  정부가 새로운 일자리를 직접 창출하는 노력도 배가하겠습니다.
sentence2:  세계에서 우리만큼 오랜 역사와 문화를 공유하는 가까운 이웃이 없습니다.
similarity:  0.0
6th d